# Create Train and Test Dataframes

In [1]:
from importlib import resources as impresources
from recurrent_health_events_prediction import configs
import os
import yaml
import pandas as pd

with open((impresources.files(configs) / 'data_config.yaml')) as f:
    data_config = yaml.safe_load(f)

DATASET = "mimic"
USE_MULTIPLE_HOSP_PATIENTS = False # Set to True to use only patients with multiple hospitalizations
SKIP_ONEHOT_ENCODING = True # Set to True to skip one-hot encoding step

training_data_config = data_config['training_data'][DATASET]

if USE_MULTIPLE_HOSP_PATIENTS:
    # Adjust path for multiple hospital patients data
    data_directory = "/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/multiple_hosp_patients"
else:
    data_directory = training_data_config['data_directory']

# Adjust path for one-hot encoded data if skipping encoding
already_encoded_data_path = "/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/train_test/more_prev_hosp_exp/train_more_prev_hosp_full.csv"

# Define output filenames, adjust if necessary
output_train_df_filename = "train_more_prev_hosp_tuning.csv"
output_test_df_filename = "validation_more_prev_hosp_tuning.csv"
full_data_path = os.path.join(data_directory, "all_events.csv")
if not SKIP_ONEHOT_ENCODING:
    print(f"Full data path: {full_data_path}")
else:
    print(f"One-hot encoded data path: {already_encoded_data_path}")


One-hot encoded data path: /workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/train_test/more_prev_hosp_exp/train_more_prev_hosp_full.csv


## Load Preprocessed Data

In [2]:
if SKIP_ONEHOT_ENCODING:
    print("Skipping one-hot encoding as per configuration.")
    df = pd.read_csv(already_encoded_data_path)
    df["ADMITTIME"] = pd.to_datetime(df["ADMITTIME"])
else:
    df = pd.read_csv(full_data_path)
    df["ADMITTIME"] = pd.to_datetime(df["ADMITTIME"])

Skipping one-hot encoding as per configuration.


In [3]:
df.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,HOSPITALIZATION_DAYS,NUM_COMORBIDITIES,TYPES_COMORBIDITIES,HAS_DIABETES,HAS_COPD,HAS_CONGESTIVE_HF,...,ADMISSION_TYPE_URGENT,INSURANCE_GOVERNMENT,INSURANCE_MEDICAID,INSURANCE_MEDICARE,INSURANCE_PRIVATE,ETHNICITY_BLACK,ETHNICITY_HISPANIC,ETHNICITY_WHITE,DISCHARGE_LOCATION_HOME,DISCHARGE_LOCATION_POST_ACUTE_CARE
0,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,8.281250,2,"['other', 'chronic_pulmonary_disease']",False,True,False,...,False,False,False,True,False,False,False,True,True,False
1,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,12.736806,2,"['other', 'chronic_pulmonary_disease']",False,True,False,...,False,False,False,True,False,False,False,True,False,True
2,107,191941,2115-02-20 17:41:00,2115-02-21 16:30:00,0.950694,2,"['other', 'renal_disease']",False,False,False,...,False,False,False,False,True,False,True,False,True,False
3,107,182383,2121-11-30 19:24:00,2121-12-05 14:18:00,4.787500,4,"['other', 'congestive_heart_failure', 'myocard...",False,False,True,...,False,False,False,True,False,False,True,False,True,False
4,124,172461,2160-06-24 21:25:00,2160-07-15 15:10:00,20.739583,3,"['other', 'chronic_pulmonary_disease', 'cerebr...",False,True,False,...,False,False,False,False,True,False,False,True,True,False


In [4]:
assert df['IN_HOSP_DEATH_EVENT'].sum().item() == 0, "There are in-hospital death events in the dataset!"
assert (df['IS_HISTORICAL_EVENT'] + df['IS_LAST_EVENT']).sum().item() != 0, "There are events that are neither historical nor last events, or both!"

## Remap Some Categorical Features

In [5]:
from recurrent_health_events_prediction.preprocessing.utils import remap_discharge_location, remap_mimic_races
if SKIP_ONEHOT_ENCODING:
    print("Skipping one-hot encoding as per configuration.")
else:
    df = remap_discharge_location(df)
    df = remap_mimic_races(df)

Skipping one-hot encoding as per configuration.


## One-Hot Encode Features

In [6]:
from recurrent_health_events_prediction.preprocessing.utils import (
    one_hot_encode_and_drop,
)
if SKIP_ONEHOT_ENCODING:
    print("Skipping one-hot encoding as per configuration.")
else:
    features_to_one_hot_encode = training_data_config["features_to_one_hot_encode"]
    one_hot_cols_to_drop = training_data_config["one_hot_cols_to_drop"]
    print("Features to one-hot encode:", features_to_one_hot_encode)
    print("One-hot columns to drop:", one_hot_cols_to_drop)
    
    df, new_cols = one_hot_encode_and_drop(
        df,
        features_to_encode=features_to_one_hot_encode,
        one_hot_cols_to_drop=one_hot_cols_to_drop,
    )
    df[new_cols].head()

Skipping one-hot encoding as per configuration.


## Split Train and Test Sets

In [7]:
from recurrent_health_events_prediction.training.split import split_train_test_stratified_group

res = split_train_test_stratified_group(
    df,
    subject_id_col="SUBJECT_ID",
    label_col="READMISSION_30_DAYS",
    charlson_col="CHARLSON_INDEX",
    test_size=0.3,
    random_state=42,
    min_stratum_size=2,
)

res.balance_report

,patients,num_visits_mean,num_visits_median,any_readmission_rate,mean_readmission_mean,mean_charlson_mean
train,1052.0,3.143536,2.0,0.500000,0.238983,5.046378
test,451.0,3.190687,2.0,0.505543,0.246209,5.047157


In [8]:
res.table.head(10)

,SUBJECT_ID,num_visits,any_readmission,mean_readmission,mean_CHARLSON_INDEX,b_num_visits,b_mean_readmission,b_mean_charlson,b_any_readm,strata
0,36,2,1,0.50,3.000000,2,"(0.3,0.6]",2-4,1,"2|(0.3,0.6]|1|2-4"
1,107,2,0,0.00,5.000000,2,0,4-6,0,2|0|0|4-6
2,124,3,0,0.00,5.333333,3,0,4-6,0,3|0|0|4-6
3,188,6,0,0.00,4.666667,6-10,0,4-6,0,6-10|0|0|4-6
4,222,4,1,0.25,7.500000,4-5,"(0.1,0.3]",6-8,1,"4-5|(0.1,0.3]|1|6-8"
5,236,2,0,0.00,3.000000,2,0,2-4,0,2|0|0|2-4
6,249,2,0,0.00,9.500000,2,0,8-10,0,2|0|0|8-10
7,291,2,0,0.00,5.000000,2,0,4-6,0,2|0|0|4-6
8,305,3,0,0.00,6.666667,3,0,6-8,0,3|0|0|6-8
9,323,3,0,0.00,6.000000,3,0,4-6,0,3|0|0|4-6


In [9]:
train_df = df[df['SUBJECT_ID'].isin(res.train_ids)]
test_df = df[df['SUBJECT_ID'].isin(res.test_ids)]

In [10]:
train_df = train_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])
test_df = test_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

In [11]:
if not SKIP_ONEHOT_ENCODING:
    train_test_data_dir = os.path.dirname(full_data_path) + "/train_test/"
else:
    train_test_data_dir = os.path.dirname(already_encoded_data_path)
os.makedirs(train_test_data_dir, exist_ok=True)
print(train_test_data_dir)

/workspaces/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/train_test/more_prev_hosp_exp


In [12]:
print("Output train_df file: ", output_train_df_filename)
print("Output test_df file: ", output_test_df_filename)

Output train_df file:  train_more_prev_hosp_tuning.csv
Output test_df file:  validation_more_prev_hosp_tuning.csv


In [13]:
print("Number of training samples: ", train_df.shape[0])
print("Number of test samples: ", test_df.shape[0])

Number of training samples:  3307
Number of test samples:  1439


In [14]:
train_df.to_csv(os.path.join(train_test_data_dir, output_train_df_filename), index=False)
test_df.to_csv(os.path.join(train_test_data_dir, output_test_df_filename), index=False)

In [15]:
train_df.head(10)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,HOSPITALIZATION_DAYS,NUM_COMORBIDITIES,TYPES_COMORBIDITIES,HAS_DIABETES,HAS_COPD,HAS_CONGESTIVE_HF,...,ADMISSION_TYPE_URGENT,INSURANCE_GOVERNMENT,INSURANCE_MEDICAID,INSURANCE_MEDICARE,INSURANCE_PRIVATE,ETHNICITY_BLACK,ETHNICITY_HISPANIC,ETHNICITY_WHITE,DISCHARGE_LOCATION_HOME,DISCHARGE_LOCATION_POST_ACUTE_CARE
0,124,172461,2160-06-24 21:25:00,2160-07-15 15:10:00,20.739583,3,"['other', 'chronic_pulmonary_disease', 'cerebr...",False,True,False,...,False,False,False,False,True,False,False,True,True,False
1,124,112906,2161-12-17 03:39:00,2161-12-24 15:35:00,7.497222,3,"['other', 'peripheral_vascular_disease', 'chro...",False,True,False,...,False,False,False,True,False,False,False,True,True,False
2,124,134369,2165-05-21 21:02:00,2165-06-06 16:00:00,15.790278,4,"['other', 'peripheral_vascular_disease', 'chro...",False,True,False,...,False,False,False,True,False,False,False,True,False,True
3,138,108120,2131-10-31 08:00:00,2131-11-06 12:54:00,6.204167,4,"['other', 'congestive_heart_failure', 'myocard...",False,True,True,...,False,False,False,False,True,False,False,True,True,False
4,138,188284,2133-12-21 23:52:00,2134-01-11 13:00:00,20.547222,3,"['other', 'congestive_heart_failure', 'maligna...",False,False,True,...,False,False,False,False,True,False,False,True,True,False
5,222,145243,2137-07-15 15:31:00,2137-07-17 12:00:00,1.853472,3,"['other', 'myocardial_infarct', 'cerebrovascul...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
6,222,105083,2141-02-18 23:10:00,2141-02-21 15:50:00,2.694444,4,"['other', 'peripheral_vascular_disease', 'myoc...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
7,222,137006,2142-06-11 07:15:00,2142-06-19 12:30:00,8.218750,7,"['myocardial_infarct', 'other', 'peripheral_va...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
8,222,188038,2142-06-23 05:02:00,2142-07-01 17:30:00,8.519444,5,"['myocardial_infarct', 'other', 'malignant_can...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
9,236,151459,2134-10-04 01:19:00,2134-10-15 16:00:00,11.611806,3,"['mild_liver_disease', 'malignant_cancer', 'ot...",False,False,False,...,False,False,False,False,True,False,False,False,True,False


## Additional metrics comparison

### Admission Type

In [16]:
train_df['ADMISSION_TYPE_ELECTIVE'].mean(), test_df['ADMISSION_TYPE_ELECTIVE'].mean()

(np.float64(0.10113137843454167), np.float64(0.11279826464208242))

### Readmission Duration Category

In [17]:
train_df['READMISSION_TIME_CAT'].value_counts(normalize=True)

READMISSION_TIME_CAT
120+      0.49619
0-30      0.25329
30-120    0.25052
Name: proportion, dtype: float64

In [18]:
test_df['READMISSION_TIME_CAT'].value_counts(normalize=True)

READMISSION_TIME_CAT
120+      0.527115
0-30      0.238612
30-120    0.234273
Name: proportion, dtype: float64